# Jane Street 2020: Multi-Layer Perceptron II

Using MLP to classify
 - Feature engineering: adding ema of features
 - Training:
   * MLP Model
   * Cost function


## 0. Summary, and initial setup

In [1]:
# Imports, environment, and paths
import os, sys, gc, random
import numpy as np
import pandas as pd
import seaborn as sns
import datatable as dtable
from sklearn.metrics import roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import GroupShuffleSplit
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
plt.style.use('dark_background') #plt.style.use('default')

# PyTorch
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
print(f'PyTorch version: {torch.__version__}; cuda available: {torch.cuda.is_available()}')

# auxiliary --------------
from importlib import reload
import time

# Print environment ------
pd.set_option('display.max_columns', 200) 
!conda info | grep 'active environment' # or use: !conda info --envs | grep '*'
print(f'working directory: {os.getcwd()}') 


PyTorch version: 1.7.1; cuda available: True
     active environment : base
working directory: /home/AWC/wang/learn/kaggle/k_JaneStreet20


In [2]:
# Reproducability
# globalSeed=67
# np.random.seed(globalSeed) # for reproducibility, does this work?

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=42)

In [3]:
# # Suppressing warning before saving the presentation, only
# import warnings
# warnings.filterwarnings('ignore')


## 1. Load data

Variables for later sections:
 - data, features: 
 - nFeat, daySet, featName

In [3]:
%%time
ddir='~/learn/kaggle/Data/JaneStreet20' # local
#ddir='../input/jane-street-market-prediction' # kaggle

# data = pd.read_csv(os.path.join(ddir,"train.csv"))
data = dtable.fread(os.path.join(ddir,"train.csv")).to_pandas() # using datatable for faster loading
features = pd.read_csv(os.path.join(ddir,"features.csv"))

nFeat=features.shape[0]
featName=[f'feature_{n}' for n in range(nFeat)]
xywCol=featName+['resp','weight']
daySet=data['date'].unique()

gc.collect();

CPU times: user 39.8 s, sys: 3.44 s, total: 43.3 s
Wall time: 3.97 s


0

## 2.  Preprecessing

The data may be used in later sections are
 - dataBlock
 - data_t, data_v, data_c for TVT data
 - nFeat, featName
 - norm, for normalization


### 2.1 Deal with nan

In [73]:
# Helper function
def fillNanWithinDay(df,dayCol,fillCol,spanFillNa=1):
    """fill NaN within date
    
    This function does (forward) fill without crossing dates, using EMA of a trailing
    window. Equal value in dayCol column indicates same date. "Date" here can
    be generalized to block with equal dayCol value
    Parameter:
      df: dataframe, original data
      dayCol: string, column name. Equal value indicates same date (block)
      fillCol: list of straings, names of columns to fill NaN
      spanFillNa: integer. Using a trailing ema of given span to fill NaN.
          spanFillNa=1 is equivalent to 'ffill' of df.fillna()
    return:
      list of pd.DataFrame of day, NaN replaced
    """
    dfList=[]
    dayList=df[dayCol].unique()
    for day in dayList:
        data_1=df.loc[df['date']==day]
        data_1_=data_1[fillCol].ewm(span=spanFillNa,adjust=False,ignore_na=True).mean()
        data_1_fill=data_1.copy()
        for cname in fillCol:
            toFill=data_1[cname].isna()
            data_1_fill.loc[toFill,cname]=data_1_.loc[toFill,cname]
        dfList.append(data_1_fill)
    
    return pd.concat(dfList)

In [4]:
data_0=data.loc[data['date']==0]

In [21]:
col=['feature_126','feature_127']
toFill=data_0[col].isna()

# toFill.idxmax()
data_0.loc[3253:3258,col]

,feature_126,feature_127
3253,4.427147,-0.248016
3254,6.196915,-1.089747
3255,3.017005,-1.155526
3256,NaN,NaN
3257,NaN,NaN
3258,NaN,NaN


In [74]:
spanFillNa=3
data_1=data_0[col].ewm(span=spanFillNa,adjust=False,ignore_na=True).mean()
data_1.loc[3253:3258]

,feature_126,feature_127
3253,3.370767,-0.300797
3254,4.783841,-0.695272
3255,3.900423,-0.925399
3256,3.900423,-0.925399
3257,3.900423,-0.925399
3258,3.900423,-0.925399


In [53]:
idx=1
print(data_1.loc[idx])
print(data_2.loc[idx])

feature_126    0.618023
feature_127    7.448875
Name: 1, dtype: float64
feature_126    0.618023
feature_127    7.448875
Name: 1, dtype: float64


In [99]:
(data_1-data_2).abs().sum(skipna=False)

feature_126    0.0
feature_127    0.0
dtype: float64

In [70]:
data_0[col].isna().sum(skipna=False)

feature_126    21
feature_127    21
dtype: int64

In [98]:
data_2.isna().sum(axis=0,skipna=False)

feature_126    0
feature_127    0
dtype: int64

In [82]:
data_2.isna().idxmax()

feature_126    3256
feature_127    3256
dtype: int64

In [86]:
data_2.loc[3258]

feature_126    0.0
feature_127    0.0
Name: 3258, dtype: float64

In [95]:
%%time
alpha=2/(spanFillNa+1)
data_2=pd.DataFrame(0,index=data_1.index,columns=data_1.columns)
for idxRow,row in data_0[col].iterrows():
    if idxRow==0:
        rowLast=row
    else:
        rowLast=(((1-alpha)*rowLast+alpha*row.fillna(0)) * (~row.isna()) + rowLast*row.isna() )
                 
    data_2.loc[idxRow]=rowLast
    if rowLast.isna().any():
        print('na'); break

CPU times: user 10.7 s, sys: 0 ns, total: 10.7 s
Wall time: 10.7 s


In [93]:
(((1-alpha)*rowLast+alpha*row.fillna(0)) * (~row.isna()) + rowLast*row.isna() )

feature_126    3.900423
feature_127   -0.925399
dtype: float64

In [30]:
%%time
# Fill NaN after first data points of day
fillCol=[f'feature_{ii}' for ii in range(nFeat) if ii not in [0,64]] # features to fill nan
data=fillNanWithinDay(data,'date',fillCol,spanFillNa=3)
print('#original: ',data.shape[0])

# Fill NaN on beginning of day
nPointStart=300 # num of samples to estimate day start
dayStart=[data.loc[data['date']==day,fillCol].iloc[:nPointStart] for day in daySet]
f_mean=pd.concat(dayStart).mean()
data[fillCol] = data[fillCol].fillna(f_mean)
f_mean.to_csv(os.path.join('./model_sv','fmean_JS20_MLP_01.csv'))

data=data.loc[~data[xywCol].isna().any(axis=1)]
data=data.loc[data['weight']>0].reset_index(drop=True)  # Dropping 0 weight
print('#After dropping NaN and 0 wieghts: ',data.shape[0])

print('#Nan in train: {:d}, {:d}, {:d}'.format(data.loc[:,featName].isna().to_numpy().sum(),
                                               data.loc[:,'resp'].isna().to_numpy().sum(),
                                               data.loc[:,'weight'].isna().to_numpy().sum()))

gc.collect()

#original:  2390491
#After dropping NaN and 0 wieghts:  1981287
#Nan in train: 0, 0, 0
CPU times: user 1min 10s, sys: 4.12 s, total: 1min 14s
Wall time: 1min 9s


0

### 2.2 Train-validation-test (TVT) split, and cast to binary classification

Cast the problem to binary classficiation proble. That is, The real-valued 'resp' is cast to {0,1}.

Split will be performed in the following way
 - Split is performed on dates
 - Reserve a latest segment of days as test
 - Train-validation split is randomized selection of days
 - In training and validation, each row is considered independent

In [7]:
# # 1.2 TVT split, and cast resp to y{0, 1}
# gss = GroupShuffleSplit(n_splits=1, test_size=0.2) #  random_state=2
# idx_t, idx_v = next(gss.split(data, groups=data['date']))


#Nan in train: 0, 0, 0
#Nan in train: 0, 0, 0


### 2.3 Centering and scaling: *not* in use

Centering and scaling is done on train set. Same scheme is applied to validation and test sets.
 - Feature 0 and 64 is not scaled


In [10]:
# Normalization
# featToNorm=[f'feature_{ii}' for ii in range(nFeat) if ii not in [0,64]]
# norm=pd.DataFrame({'c':pd.Series(0,index=featName), 's':pd.Series(1,index=featName) }).T
# norm.loc['c',featToNorm]=data_t['X'][featToNorm].mean()
# norm.loc['s',featToNorm]=data_t['X'][featToNorm].std()

# data_t['X']=(data_t['X']-norm.loc['c'])/norm.loc['s']
# data_v['X']=(data_v['X']-norm.loc['c'])/norm.loc['s']


In [11]:
# Verify proper normalization
# _, axs = plt.subplots(nrows=1, ncols=2,figsize=(16, 3))
# data_t['X'][featToNorm].mean().plot(ax=axs[0])
# data_v['X'][featToNorm].mean().plot(ax=axs[0])
# axs[0].set(ylabel='mean')
# data_t['X'][featToNorm].std().plot(ax=axs[1])
# data_v['X'][featToNorm].std().plot(ax=axs[1])
# axs[1].set(ylabel='std');

## 3. Traing MLP

### 3.1 Model

In [31]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = torch.nn.BatchNorm1d(len(features))
        self.dropout0 = torch.nn.Dropout(0.10143786981358652)

        hidden_size = 256
        self.dense1 = torch.nn.Linear(len(features), 384)
        self.batch_norm1 = torch.nn.BatchNorm1d(384)
        self.dropout1 = torch.nn.Dropout(0.19720339053599725)

        self.dense2 = torch.nn.Linear(384, 896)
        self.batch_norm2 = torch.nn.BatchNorm1d(896)
        self.dropout2 = torch.nn.Dropout(0.2703017847244654)

        self.dense3 = torch.nn.Linear(896, 896)
        self.batch_norm3 = torch.nn.BatchNorm1d(896)
        self.dropout3 = torch.nn.Dropout(0.23148340929571917)

        self.dense4 = torch.nn.Linear(896, 394)
        self.batch_norm4 = torch.nn.BatchNorm1d(394)
        self.dropout4 = torch.nn.Dropout(0.2357768967777311)

        self.dense5 = torch.nn.Linear(394, 1)

        self.Relu = torch.nn.ReLU(inplace=True)
        self.PReLU = torch.nn.PReLU()
        self.LeakyReLU = torch.nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = torch.nn.GELU()
        self.RReLU = torch.nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x = self.dense1(x)
        x = self.batch_norm1(x)
        x = x * torch.sigmoid(x)
        x = self.dropout1(x)

        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = x * torch.sigmoid(x)
        x = self.dropout2(x)
        
        x = self.dense3(x)
        x = self.batch_norm3(x)
        x = x * torch.sigmoid(x)
        x = self.dropout3(x)
        
        x = self.dense4(x)
        x = self.batch_norm4(x)
        x = x * torch.sigmoid(x)
        x = self.dropout4(x)

        x = self.dense5(x)

        return x

### 3.2 Trainging: helper functions

In [32]:
class MarketDataset:
    def __init__(self, data, featName):
        self.features = data[featName].values
        self.label = (data['resp']>0).astype('int').values.reshape(-1, 1)
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.label[idx], dtype=torch.float)
        }

In [33]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for data in dataloader:
        optimizer.zero_grad()
        features = data['features'].to(device)
        label = data['label'].to(device)
        outputs = model(features)
        loss = loss_fn(outputs, label)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)

    return final_loss

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        features = data['features'].to(device)

        with torch.no_grad():
            outputs = model(features)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds).reshape(-1)

    return preds

In [34]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [35]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [36]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    # print('weight: ', weight)
    # print('resp: ', resp)
    # print('action: ', action)
    # print('weight * resp * action: ', weight * resp * action)
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

### 3.3 Training

In [37]:
batch_size = 4096
label_smoothing = 1e-2
learning_rate = 1e-3

start_time = time.time()

gss = GroupShuffleSplit(n_splits=1, test_size=0.2) #  random_state=2
idx_t, idx_v = next(gss.split(data, groups=data['date']))

train_set = MarketDataset(data.loc[idx_t],featName)
train_loader=DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_set = MarketDataset(data.loc[idx_v],featName)
valid_loader=DataLoader(valid_set, batch_size=batch_size, shuffle=False) # Using True is bad, why??????????

torch.cuda.empty_cache()
device = torch.device("cuda:0")
model = Model()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = SmoothBCEwLogits(smoothing=label_smoothing)

ckp_path = os.path.join('./model_sv','JS20_MLP_01.pth')

es = EarlyStopping(patience=3, mode="max")
for epoch in range(10):
    train_loss = train_fn(model, optimizer, None, loss_fn, train_loader, device)
    valid_pred = inference_fn(model, valid_loader, device)
    auc_score = roc_auc_score((data.loc[idx_v,'resp']>0).astype(int).values.reshape(-1, 1), valid_pred)
    logloss_score = log_loss((data.loc[idx_v,'resp']>0).astype(int).values.reshape(-1, 1), valid_pred)
    valid_pred = np.where(valid_pred >= 0.5, 1, 0).astype(int)

    u_score = utility_score_bincount(date=data.loc[idx_v,'date'].values, weight=data.loc[idx_v,'weight'].values,
                                     resp=data.loc[idx_v,'resp'].values, action=valid_pred)
    print(f"EPOCH:{epoch:3}, train_loss:{train_loss:.5f}, u_score:{u_score:.5f}, auc:{auc_score:.5f}, logloss:{logloss_score:.5f}, "
          f"time: {(time.time() - start_time) / 60:.2f}min")

    es(auc_score, model, model_path=ckp_path)
    if es.early_stop:
        print("Early stop!")
        break
#     break # only train 1 model for fast, you can remove it to train 5 folds

EPOCH:  0, train_loss:0.69388, u_score:1246.02201, auc:0.52964, logloss:0.69140, time: 0.95min
Validation score improved (-inf --> 0.529644626795742). Saving model!
EPOCH:  1, train_loss:0.69082, u_score:1487.25101, auc:0.53333, logloss:0.69140, time: 1.89min
Validation score improved (0.529644626795742 --> 0.5333268874827157). Saving model!
EPOCH:  2, train_loss:0.69010, u_score:1513.15369, auc:0.53031, logloss:0.69197, time: 2.82min
EarlyStopping counter: 1 out of 3
EPOCH:  3, train_loss:0.68933, u_score:834.25969, auc:0.53124, logloss:0.69228, time: 3.77min
EarlyStopping counter: 2 out of 3
EPOCH:  4, train_loss:0.68840, u_score:1589.11385, auc:0.53404, logloss:0.69219, time: 4.72min
Validation score improved (0.5333268874827157 --> 0.5340436295219545). Saving model!
EPOCH:  5, train_loss:0.68743, u_score:1734.98346, auc:0.53099, logloss:0.69339, time: 5.62min
EarlyStopping counter: 1 out of 3
EPOCH:  6, train_loss:0.68666, u_score:796.98737, auc:0.52877, logloss:0.69546, time: 6.51

## 4. Predict on test set

 - o wieght rows does not appear to have much impact (small, worse)

In [ ]:
# Load Model
# Load test set
# Predict in a loop

In [20]:
# EPOCH:  0, train_loss:0.69401, u_score:1256.95984, auc:0.52760, logloss:0.69205, time: 1.12min
# Validation score improved (-inf --> 0.5276042643733097). Saving model!
# EPOCH:  1, train_loss:0.69051, u_score:1226.78192, auc:0.52978, logloss:0.69214, time: 2.19min
# Validation score improved (0.5276042643733097 --> 0.5297800412247824). Saving model!
# EPOCH:  2, train_loss:0.68943, u_score:968.38580, auc:0.52706, logloss:0.69382, time: 3.26min
# EarlyStopping counter: 1 out of 3
# EPOCH:  3, train_loss:0.68845, u_score:972.59923, auc:0.52657, logloss:0.69465, time: 4.35min
# EarlyStopping counter: 2 out of 3
# EPOCH:  4, train_loss:0.68738, u_score:1267.13899, auc:0.52882, logloss:0.69441, time: 5.49min
# EarlyStopping counter: 3 out of 3
# Early stop!


# Run 1:
# EPOCH:  0, train_loss:0.69384, u_score:617.65718, auc:0.52557, logloss:0.69230, time: 0.86min
# Validation score improved (-inf --> 0.5255701715967822). Saving model!
# EPOCH:  1, train_loss:0.69121, u_score:545.33534, auc:0.53062, logloss:0.69306, time: 1.72min
# Validation score improved (0.5255701715967822 --> 0.5306179912922396). Saving model!
# EPOCH:  2, train_loss:0.69048, u_score:526.00783, auc:0.52900, logloss:0.69320, time: 2.57min
# EarlyStopping counter: 1 out of 3
# EPOCH:  3, train_loss:0.68964, u_score:528.03468, auc:0.53532, logloss:0.69383, time: 3.43min
# Validation score improved (0.5306179912922396 --> 0.5353218402403408). Saving model!
# EPOCH:  4, train_loss:0.68918, u_score:542.65725, auc:0.53162, logloss:0.69245, time: 4.29min
# EarlyStopping counter: 1 out of 3
# EPOCH:  5, train_loss:0.68864, u_score:155.54247, auc:0.52435, logloss:0.69569, time: 5.15min
# EarlyStopping counter: 2 out of 3
# EPOCH:  6, train_loss:0.68809, u_score:374.29251, auc:0.52604, logloss:0.69363, time: 6.01min
# EarlyStopping counter: 3 out of 3
# Early stop!
